In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io
from config import AWS

In [3]:
#connecting to postgres DB in AWS RDS to create a dataFrame
engine = create_engine(AWS)
df = pd.read_sql('bls_jobs_2020', engine)
df.head()

,index,area,area_title,prim_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000
0,5,10180,"Abilene, TX",TX,11-3021,Computer and Information Systems Managers,70.0,30.2,1.054
1,29,10180,"Abilene, TX",TX,15-0000,Computer and Mathematical Occupations,820.0,15.3,12.372
2,30,10180,"Abilene, TX",TX,15-1211,Computer Systems Analysts,110.0,23.0,1.674
3,31,10180,"Abilene, TX",TX,15-1232,Computer User Support Specialists,220.0,14.5,3.374
4,32,10180,"Abilene, TX",TX,15-1244,Network and Computer Systems Administrators,150.0,28.1,2.227


In [8]:
city_df = df[['index','area', 'area_title','prim_state','occ_title',]]
city_df.head()

,index,area,area_title,prim_state,occ_title
0,5,10180,"Abilene, TX",TX,Computer and Information Systems Managers
1,29,10180,"Abilene, TX",TX,Computer and Mathematical Occupations
2,30,10180,"Abilene, TX",TX,Computer Systems Analysts
3,31,10180,"Abilene, TX",TX,Computer User Support Specialists
4,32,10180,"Abilene, TX",TX,Network and Computer Systems Administrators


In [5]:
ml_df = df.drop(['area_title','prim_state','occ_title'], axis=1)
ml_df.head()

,index,area,occ_code,tot_emp,emp_prse,jobs_1000
0,5,10180,11-3021,70.0,30.2,1.054
1,29,10180,15-0000,820.0,15.3,12.372
2,30,10180,15-1211,110.0,23.0,1.674
3,31,10180,15-1232,220.0,14.5,3.374
4,32,10180,15-1244,150.0,28.1,2.227


In [6]:
ml_df['occ_code'] = df['occ_code'].astype(str).apply(lambda x: x.replace('-', '')).astype(int)
ml_df.head()

,index,area,occ_code,tot_emp,emp_prse,jobs_1000
0,5,10180,113021,70.0,30.2,1.054
1,29,10180,150000,820.0,15.3,12.372
2,30,10180,151211,110.0,23.0,1.674
3,31,10180,151232,220.0,14.5,3.374
4,32,10180,151244,150.0,28.1,2.227


In [7]:
ml_df.dtypes

index          int64
area           int64
occ_code       int32
tot_emp      float64
emp_prse     float64
jobs_1000    float64
dtype: object

In [9]:
#connecting to postgres DB to create 2 new tables, area_code and city_data 
engine = create_engine(AWS)
#use this table to use for ML input of jobs, occ_code is not an INT and all catagorial data removed
ml_df.to_sql('area_code', engine, if_exists='replace')
#use this table for city data, both tables have index and area for forgien keys. 
city_df.to_sql('city_data', engine, if_exists='replace')